<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mecab-python3
!pip install mojimoji

     |████████████████████████████████| 15.9MB 28.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
import re
import MeCab
import mojimoji

def serialize(words):
    #大文字統一
    words = words.upper()

    #チルダ統一
    ser_word = ["〜","～", "-","－","から","MAX","~~"]
    for word in ser_word:
      words = words.replace(word,"~")
    
    #スキル見合い
    ser_word = ["スキル見合い","スキル見合","ご相談","相談"]
    for word in ser_word:
      words = words.replace(word,"00")

    #改行コード変換
    words = words.replace("\r\n","")
      
    #万円単位変換
    words = words.replace("0000","")
      
    #英数字のみ半角統一
    words = mojimoji.zen_to_han(words, kana=False)
      
    return words

In [0]:
def tanka_pickup(str_wk):
  #======================================================
  #スキップワード削除
  #======================================================
  del_word = ["<",">","＜","＞","[","]","《","》","≪","≫","【", "】","(",")","（","）"," ","　",":","：","完全","位","くらい","ぐらい","およそ","大体","だいたい","約","前後","円","万","下さい","ください"]
  
  base_word = '単価'
  tanka_yuragi = ["単金","金額","価格","月額","予算"]

  #◆単価が含まれるであろう箇所だけ渡す
  #正規化
  str_wk = serialize(str_wk)

  #削除
  for word in del_word:
    str_wk = str_wk.replace(word,'')

  #ゆらぎ統一
  for tanka in tanka_yuragi:
    str_wk = str_wk.replace(tanka,base_word)

  #======================================================
  #分かち書き
  #======================================================
  tagger = MeCab.Tagger("-Owakati")
  list_output = (tagger.parse(str_wk)).split(' ')

  #======================================================
  #「単価」に最も近い位置で書かれている数字が含まれる文節を抽出
  #======================================================
  tanka_str = ""

  #上限範囲指定
  if list_output.count(base_word) == 1:
    if list_output[list_output.index(base_word)+1] == "~":
      tanka_str = "00~" + list_output[list_output.index(base_word)+2]

  #上限下限指定
  if list_output.count(base_word) == 1:
    if list_output[list_output.index(base_word)+2] == "~":
      tanka_str = tanka_str + "~" + list_output[list_output.index(base_word)+3]

  #上限のみ指定
  #★エラー有り
  #if list_output[list_output.index(base_word)+1] == "MAX":
  #  tanka_str ="00~" + list_output[list_output.index(base_word)+2]
    
  if list_output.count(base_word) > 0:
    if list_output[list_output.index(base_word)+1].isdecimal() is True or list_output[list_output.index(base_word)+2].isdecimal() is True or list_output[list_output.index(base_word)+3].isdecimal() is True:
      tanka_str = list_output[list_output.index(base_word)+1] + "/" + list_output[list_output.index(base_word)+2] + "/" + list_output[list_output.index(base_word)+3] + "/" +list_output[list_output.index(base_word)+4]
    else:
      tanka_str = "単価情報あり"
  else:
    tanka_str = "単価完全なし"
    
  return tanka_str

In [15]:
import pandas as pd
import codecs

#utf-8で作られていないファイルの場合のCSV読み込み方法
with codecs.open("python.CSV", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")

df2 = df[["本文"]]

#行単位の繰り返し
for index,item in df2.iterrows():
  str_wk = item['本文']
  pickup_str = tanka_pickup(str_wk)
  print(str(index) + ":" + pickup_str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


0:~/55///月
1:単価完全なし
2:単価完全なし
3:単価完全なし
4:単価情報あり
5:65/◆/枠/数
6:~/85140/H/~
7:80/./0/まで
8:単価/80///月
9:~/90140/H/~
10:60/~/80/精算
11:単価完全なし
12:~/75140/H/~
13:85/◆/面談/2
14:~/95140/H/~
15:~/95140/H/~
16:00/できる/方/で
17:~/70140/H/~
18:単価/80///月
19:60/~/80/精算
20:00/できる/方/で
21:~/85140/H/~
22:65/◆/枠/数
23:~/90140/H/~
24:~/90140/H/~
25:80/~/90/程度
26:単価/80///月
27:00/※/PYTHON/エンジニア
28:85/◆/面談/2
29:~/95140/H/~
30:~/95140/H/~
31:~/75140/H/~
32:~/70140/H/~
33:80/~/90/程度
34:85/◆/面談/2
35:00/※/PYTHON/エンジニア
36:~/95140/H/~
37:~/95140/H/~
38:単価情報あり
39:~/75140/H/~
40:90/./0/まで
41:~/70140/H/~
42:85/./0/まで
43:54/業種/職務/システムエンジニア
44:単価完全なし
45:00/※/PYTHON/エンジニア
46:~/85140/H/~
47:65/◆/枠/数
48:~/7500/精算/固定
49:00/■/精算/140
50:80/./0/まで
51:~/90140/H/~
52:単価情報あり
53:60/~/80/精算
54:85/◆/面談/2
55:~/95140/H/~
56:~/95140/H/~
57:単価/65/~/75
58:~/75140/H/~
59:~/70140/H/~
60:~~/150140/H/~
61:65/◆/枠/数
62:60/~/80/※
63:60/~/80/精算
64:単価/70/~※/ご
65:00/※/ご/希望
66:85/◆/面談/2
67:60/~/80/※
68:00/できる/方/で
69:~/75140/H/~
70:~/70140/H/~
71:90/./0/まで


In [0]:
print(df2.iloc[166])

本文    \r\n\r\n-----Original Message-----\r\nFrom: 【ス...
Name: 166, dtype: object


In [0]:
#単価変数初期化
min_tanka = 0
max_tanka = 0
med_tanka = 0

#======================================================
#範囲指定単価(exp:A～B) 編集
#======================================================
tanka_range = re.findall('[0-9]{2}~[0-9]{2}',pickup_str)

#中間値算出
#if len(tanka_range) == 1:
#  tanka1 = int(tanka_range[0].split('~')[0])
#  tanka2 = int(tanka_range[0].split('~')[1])
#  med_tanka = round((tanka1+tanka2)/2)
#else:
#  med_tanka = "0"

#下限算出
if len(tanka_range) == 1:
  min_tanka = int(tanka_range[0].split('~')[0])
else:
  min_tanka = 0

#上限算出
if len(tanka_range) == 1:
  max_tanka = int(tanka_range[0].split('~')[1])
else:
  max_tanka = 0

print("MIN:" + str(min_tanka))
print("MAX:" + str(max_tanka))


00~55
MIN:0
MAX:55
